In [1]:
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import tensorflow as tf
import deeppavlov
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-08-15 15:35:46.991 DEBU

In [2]:
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import FewshotNerBinaryClassifier, CompositeEmbedder
from src.fewshot_ner_viz_component.data_processing import *

### Configuration parameters

In [3]:
NE_TYPES = ['PERSON']
NE_TYPES = ['PERSON', 'GPE', 'ORG', 'LOC']
N_EXAMPLES = 10
N_TEST_SENTENCES = 100
N_SAMPLES = 10
MODELS = ['ne_centroid', 'ne_nearest', 'weighted_kNN', 'centroid_kNN', 'svm']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}

In [4]:
dataset_orig = read_data()

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [5]:
embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=True, use_glove=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-08-15 15:35:54.352 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


2018-08-15 15:35:54.710 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights


...

2018-08-15 15:35:54.953 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel


INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


2018-08-15 15:35:54.960 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


2018-08-15 15:35:54.966 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed
2018-08-15 15:35:55.440 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 60: [loading embeddings from `/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt`]
2018-08-15 15:35:55.441 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 204: loading projection weights from /home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt
2018-08-15 15:35:55.441 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt'}
2018-08-15 15:36:24.260 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 266: loaded (400000, 100) matr

In [6]:
from collections import defaultdict
f1_scores = {}
for ne_type in NE_TYPES:
    print('NE type: {}'.format(ne_type))
    dataset = filter_dataset_by_ne_types(dataset_orig, ne_type)
    f1_scores[ne_type] = {}
    for model in MODELS:
        f1_scores[ne_type][model] = {'values':[]}
    for idx_sample in range(N_SAMPLES):
        print('Sample {}'.format(idx_sample+1))
        tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
        tokens_test,tags_test = get_data_sample(dataset['test'], N_TEST_SENTENCES)
        y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
        ner_bin_clf = FewshotNerBinaryClassifier(embedder)
        ner_bin_clf.train_on_batch(tokens_train, tags_train)
        results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
        pred_avg = np.zeros(y_test.size)
        n_models_avg = 0
        for model in MODELS:
            pred = results[model]['pred']
            if model in ['ne_centroid', 'weighted_kNN', 'centroid_kNN', 'svm']:   
                pred_avg += pred
                n_models_avg += 1
            f1 = f1_score(y_test, pred)
            f1_scores[ne_type][model]['values'].append(f1)
        pred_avg /= n_models_avg
        
        cur_f1_scores_dict = {k: f1_scores[ne_type][k]['values'][-1] for k in f1_scores[ne_type].keys()}
        print('F1 scores: {}'.format(cur_f1_scores_dict))
    for model in MODELS:
        f1_scores[ne_type][model]['mean'] = np.mean(f1_scores[ne_type][model]['values'])
        f1_scores[ne_type][model]['std'] = np.std(f1_scores[ne_type][model]['values'])
        print('{} F1: mean = {} std = {}'.format(model, f1_scores[ne_type][model]['mean'], f1_scores[ne_type][model]['std']))

NE type: PERSON
Num of train sentences: 12195
Num of valid sentences: 1553
Num of test sentences: 1573
Sample 1
Train
NE centroid similarity model
NE nearest similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
F1 scores: {'ne_centroid': 0.8144578313253013, 'ne_nearest': 0.32214765100671144, 'weighted_kNN': 0.8912133891213389, 'centroid_kNN': 0.8714285714285714, 'svm': 0.8806941431670282}
Sample 2
Train
NE centroid similarity model
NE nearest similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
F1 scores: {'ne_centroid': 0.562874251497006, 'ne_nearest': 0.3443708609271523, 'weighted_kNN': 0.8747433264887065, 'centroid_kNN': 0.8200000000000001, 'svm': 0.8733031674208145}
Sample 3
Train
NE centroi

NE centroid similarity model
NE nearest similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
F1 scores: {'ne_centroid': 0.7089337175792507, 'ne_nearest': 0.33198380566801616, 'weighted_kNN': 0.6666666666666666, 'centroid_kNN': 0.7376623376623376, 'svm': 0.7342995169082126}
Sample 10
Train
NE centroid similarity model
NE nearest similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
F1 scores: {'ne_centroid': 0.7331670822942644, 'ne_nearest': 0.35146443514644354, 'weighted_kNN': 0.7548076923076923, 'centroid_kNN': 0.8208092485549133, 'svm': 0.8682170542635659}
ne_centroid F1: mean = 0.7365936417695736 std = 0.02964126410506379
ne_nearest F1: mean = 0.45583844070161933 std = 0.08964473748151745
weig

NE centroid similarity model
NE nearest similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
F1 scores: {'ne_centroid': 0.5598958333333334, 'ne_nearest': 0.4078624078624079, 'weighted_kNN': 0.6625000000000001, 'centroid_kNN': 0.6229508196721312, 'svm': 0.6553191489361703}
Sample 7
Train
NE centroid similarity model
NE nearest similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
F1 scores: {'ne_centroid': 0.6178623718887262, 'ne_nearest': 0.3890577507598784, 'weighted_kNN': 0.5134649910233393, 'centroid_kNN': 0.6863157894736842, 'svm': 0.5897035881435256}
Sample 8
Train
NE centroid similarity model
NE nearest similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[========

### Print results

In [7]:
print('{:<8s}\t{:<8s}\t{:<8s}'.format('MODEL', 'MEAN', 'STD'))
for ne_type in NE_TYPES:
    print('---------------{}---------------'.format(ne_type))
    for model in MODELS:
        print('{:<8s}\t{:<8.2f}\t{:<8.2f}'.format(model, f1_scores[ne_type][model]['mean']*100, f1_scores[ne_type][model]['std']*100))

MODEL   	MEAN    	STD     
---------------PERSON---------------
ne_centroid	67.36   	13.30   
ne_nearest	24.72   	9.11    
weighted_kNN	84.73   	4.09    
centroid_kNN	79.87   	9.20    
svm     	86.07   	2.44    
---------------GPE---------------
ne_centroid	73.66   	2.96    
ne_nearest	45.58   	8.96    
weighted_kNN	72.28   	2.74    
centroid_kNN	73.89   	5.30    
svm     	78.51   	4.28    
---------------ORG---------------
ne_centroid	58.19   	4.09    
ne_nearest	19.79   	3.57    
weighted_kNN	63.25   	5.81    
centroid_kNN	46.52   	18.00   
svm     	73.79   	3.57    
---------------LOC---------------
ne_centroid	60.43   	5.08    
ne_nearest	46.49   	5.99    
weighted_kNN	62.46   	5.58    
centroid_kNN	62.21   	4.52    
svm     	67.23   	3.88    
